# Chains 종류

**LangChain**을 사용하여 다양한 체인(Chain) 및 LLM(대규모 언어 모델) 기반 애플리케이션을 구축합니다.

**1. Simple Chain (단일 체인)**  
- 하나의 프롬프트를 통해 LLM(OpenAI)을 사용하여 텍스트를 생성합니다.  


**2. Simple Sequential Chain (연속 체인)**  
-  여러 LLM 호출을 연속적으로 수행하여 출력을 다음 입력으로 전달합니다.  

**3. Document 요약 체인**  
- **목적:** 텍스트 문서를 요약합니다.  

**4. 텍스트를 Vector Store로 변환**  
- **4.1 VectortstoreIndexCreator**   
- **4.2 Chroma DB 사용**  

**5. HTTP Request Chain (웹 요청 체인)**  
- HTTP 요청을 통해 외부 웹 데이터에서 정보를 추출합니다.
  

In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv()) # read local .env file

True

In [2]:
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"

## 1. Simple Chain (단일 체인)
- 가장 기본적인 유형의 체인
- 입력 프롬프트를 수신하고 이를 사용하여 텍스트를 생성하는 역할을 담당하는 언어 모델(LLM) 하나만 있습니다.

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

llm = ChatOpenAI(model="gpt-4o-mini")

- Prompt Template 작성

In [4]:
prompt = PromptTemplate(
    input_variables=["place"],
    template="{place}에서 가장 가 볼만한 곳은?",
)

In [18]:
chain = prompt | llm
# chain = prompt.pipe(llm)

# 입력 변수만 지정하여 체인을 실행
print(chain.invoke("한국").content)

한국에는 다양한 매력적인 관광지가 있습니다. 몇 가지 추천할 만한 곳은 다음과 같습니다:

1. **서울**: 경복궁, 북촌 한옥마을, 명동, 인사동 등 역사와 현대가 어우러진 곳입니다.
2. **부산**: 해운대, 광안리, 태종대 등 아름다운 해변과 활기찬 시장이 매력적입니다.
3. **제주도**: 한라산, 성산일출봉, 만장굴 등 자연 경관이 뛰어나고, 독특한 문화와 음식도 즐길 수 있습니다.
4. **경주**: 불국사, 석굴암, 안압지 등 고대 신라의 역사와 문화를 느낄 수 있는 곳입니다.
5. **전주**: 전주 한옥마을에서 전통 한옥을 감상하고, 비빔밥과 같은 맛있는 음식을 즐길 수 있습니다.
6. **강릉**: 아름다운 해변과 커피 거리, 오죽헌 등 자연과 문화를 동시에 즐길 수 있습니다.

각 지역마다 특색이 있으니, 관심 있는 주제나 활동에 따라 선택해 보세요!


## 2. Simple Sequential Chains (연속 체인)
- Sequential Chain은 언어 모델에 대한 일련의 연속 호출 포함
- 이 접근 방식은 한 호출에서 생성된 출력을 다른 호출의 입력으로 활용할 때 특히 유용

- Prompt Template 작성

In [19]:
template_1 = """{place}에서 방문하기 가장 좋은 장소 5곳을 추천해주세요

응답:
"""
prompt1 = PromptTemplate(
    input_variables=["place"], 
    template=template_1)

chain_1 = prompt1 | llm

In [20]:
template_2 = """장소 목록이 주어지면, 모든 장소를 방문하는 데 드는 비용과 방문에 필요한 날짜를 현지 통화로 추산해 주십시오. 
그리고 나서 예산 {budget}과 비교하여 충분한지, 부족한지 계산해 주세요.

응답:
"""
prompt2 = PromptTemplate(
    input_variables=["budget"],
    template=template_2)

chain_2 = prompt2 | llm

In [21]:
final_chain = chain_1 | chain_2
final_chain

PromptTemplate(input_variables=['place'], input_types={}, partial_variables={}, template='{place}에서 방문하기 가장 좋은 장소 5곳을 추천해주세요\n\n응답:\n')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001BC351221F0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001BC351303A0>, root_client=<openai.OpenAI object at 0x000001BC35023B20>, root_async_client=<openai.AsyncOpenAI object at 0x000001BC35122250>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'))
| PromptTemplate(input_variables=['budget'], input_types={}, partial_variables={}, template='장소 목록이 주어지면, 모든 장소를 방문하는 데 드는 비용과 방문에 필요한 날짜를 현지 통화로 추산해 주십시오. \n그리고 나서 예산 {budget}과 비교하여 충분한지, 부족한지 계산해 주세요.\n\n응답:\n')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001BC351221F0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001BC351303A0>, root_client=<openai.OpenAI object at 0x000001BC35023

In [22]:
review = final_chain.invoke({"place": "한국", "budget": "1,000,000"})
review

AIMessage(content='각 장소를 방문하는 데 드는 비용과 필요한 날짜를 대략적으로 추산해 보겠습니다. 이를 위해 각 장소의 평균적인 여행 비용과 방문 기간을 고려하겠습니다.\n\n1. **서울 경복궁**\n   - 비용: 약 2만 원 (입장료, 교통비 포함)\n   - 소요 기간: 1일\n\n2. **부산 해운대 해수욕장**\n   - 비용: 약 5만 원 (교통비, 해변 이용비 포함)\n   - 소요 기간: 1일\n\n3. **제주도**\n   - 비용: 약 30만 원 (항공료, 숙박비, 음식비 포함)\n   - 소요 기간: 3일\n\n4. **경주**\n   - 비용: 약 3만 원 (교통비, 입장료 포함)\n   - 소요 기간: 1일\n\n5. **전주 한옥마을**\n   - 비용: 약 3만 원 (교통비, 음식비 포함)\n   - 소요 기간: 1일\n\n### 총 비용과 소요 기간\n- **총 비용**: 2만 원 (경복궁) + 5만 원 (해운대) + 30만 원 (제주도) + 3만 원 (경주) + 3만 원 (전주) = **43만 원**\n- **총 소요 기간**: 1일 (경복궁) + 1일 (해운대) + 3일 (제주도) + 1일 (경주) + 1일 (전주) = **7일**\n\n### 예산 검토\n추천한 장소의 방문을 위한 총 비용은 **43만 원**이며, 소요 기간은 **7일**입니다. 이 예산이 충분한지, 부족한지는 개인의 여행 스타일, 숙박 선택, 식사 수준에 따라 달라질 수 있습니다. \n\n- **충분한 경우**: 저렴한 숙소를 선택하거나, 대중교통을 이용해 비용을 절약하는 경우.\n- **부족한 경우**: 고급 숙소에 머물거나, 외식을 자주 하는 경우.\n\n여행 계획에 맞춰 예산을 조정하는 것이 좋습니다! 추가적인 질문이나 세부 정보가 필요하시면 말씀해 주세요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 505, 

In [23]:
print(review.content)

각 장소를 방문하는 데 드는 비용과 필요한 날짜를 대략적으로 추산해 보겠습니다. 이를 위해 각 장소의 평균적인 여행 비용과 방문 기간을 고려하겠습니다.

1. **서울 경복궁**
   - 비용: 약 2만 원 (입장료, 교통비 포함)
   - 소요 기간: 1일

2. **부산 해운대 해수욕장**
   - 비용: 약 5만 원 (교통비, 해변 이용비 포함)
   - 소요 기간: 1일

3. **제주도**
   - 비용: 약 30만 원 (항공료, 숙박비, 음식비 포함)
   - 소요 기간: 3일

4. **경주**
   - 비용: 약 3만 원 (교통비, 입장료 포함)
   - 소요 기간: 1일

5. **전주 한옥마을**
   - 비용: 약 3만 원 (교통비, 음식비 포함)
   - 소요 기간: 1일

### 총 비용과 소요 기간
- **총 비용**: 2만 원 (경복궁) + 5만 원 (해운대) + 30만 원 (제주도) + 3만 원 (경주) + 3만 원 (전주) = **43만 원**
- **총 소요 기간**: 1일 (경복궁) + 1일 (해운대) + 3일 (제주도) + 1일 (경주) + 1일 (전주) = **7일**

### 예산 검토
추천한 장소의 방문을 위한 총 비용은 **43만 원**이며, 소요 기간은 **7일**입니다. 이 예산이 충분한지, 부족한지는 개인의 여행 스타일, 숙박 선택, 식사 수준에 따라 달라질 수 있습니다. 

- **충분한 경우**: 저렴한 숙소를 선택하거나, 대중교통을 이용해 비용을 절약하는 경우.
- **부족한 경우**: 고급 숙소에 머물거나, 외식을 자주 하는 경우.

여행 계획에 맞춰 예산을 조정하는 것이 좋습니다! 추가적인 질문이나 세부 정보가 필요하시면 말씀해 주세요.


## Text를 Vector Store로 변환

텍스트 임베딩을 생성하는 모델을 준비했다면, 이제 이를 효율적인 **유사도 검색(Similarity Search)** 을 지원하는 **벡터 스토어(Vector Stores)** 에 저장할 수 있습니다.

## **벡터 스토어(Vector Stores)**  

LangChain의 [**VectorStore**](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStore.html) 객체는 **텍스트 및 `Document` 객체를 저장**하고, 다양한 **유사도 메트릭(Similarity Metrics)** 을 사용해 쿼리를 수행할 수 있는 메서드를 포함합니다. 

In [24]:
from langchain_openai import OpenAIEmbeddings

# 사용할 임베딩 모델의 이름을 지정
embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

In [25]:
from langchain_core.documents import Document

# 텍스트 파일 읽기
with open('TEXT/sample3_kor.txt', encoding="utf-8") as f:
    text = f.read()

# 원본 문서를 Document 객체로 변환
docs = Document(page_content=text)
docs

Document(metadata={}, page_content='제목: 컴퓨터: 기술 세계를 혁신하다\n\n서론:\n컴퓨터는 수많은 방식으로 세상을 혁신한 인간의 독창성의 경이로움입니다. 복잡한 계산기라는 겸손한 시작에서 우리 삶의 모든 측면에서 유비쿼터스 도구로서의 현재 지위에 이르기까지 컴퓨터는 우리가 일하고, 소통하고, 배우고, 즐기는 방식을 변화시켰습니다. 이 에세이는 우리의 현대 세계를 형성하는 컴퓨터의 진화, 영향 및 미래 잠재력을 탐구합니다.\n\n컴퓨터의 탄생:\n컴퓨터는 발명가와 수학자들이 복잡한 계산을 자동화할 수 있는 기계를 개념화하기 시작한 19세기 초반에 시작되었습니다. 그러나 최초의 전자 디지털 컴퓨터가 개발된 것은 20세기 중반이었습니다. 앨런 튜링, 존 폰 노이만, 그레이스 호퍼와 같은 선구자들은 이 분야에 상당한 공헌을 했으며 오늘날 우리가 알고 있는 컴퓨터의 토대를 마련했습니다.\n\n컴퓨팅 파워의 진화:\n방 크기의 메인프레임에서 휴대용 노트북에 이르기까지 컴퓨터는 크기, 속도, 처리 능력 면에서 기하급수적으로 진화했습니다. 집적 회로, 마이크로프로세서, 마이크로칩의 트랜지스터 수가 약 2년마다 두 배가 된다는 무어의 법칙이 도입되면서 컴퓨터 기술이 발전했습니다. 이러한 기하급수적 성장으로 인해 더 빠르고 효율적이며 점점 더 강력한 장치가 개발되어 계속해서 세상을 재편하고 있습니다.\n\n업무와 생산성의 변화:\n컴퓨터는 산업 전반에 걸쳐 생산성, 효율성, 정확성을 높여 직장을 변화시켰습니다. 반복적인 작업을 자동화하고 운영을 간소화하며 글로벌 연결을 용이하게 했습니다. 워드 프로세싱과 데이터 분석에서 복잡한 시뮬레이션과 인공 지능에 이르기까지 컴퓨터는 금융, 의료, 엔지니어링, 창작 예술과 같은 분야의 전문가에게 필수적인 도구가 되었습니다. 원격 작업과 디지털 협업의 출현은 현대 작업 환경에 미치는 컴퓨터의 영향을 더욱 강조합니다.\n\n통신 및 연결성:\n컴퓨터는 통신에 혁명을 일으켜 사람들이 먼 거리에 있는 사람들과 연결

In [26]:
# RecursiveCharacterTextSplitter를 가져옵니다.
# 이 도구는 텍스트를 재귀적으로 분할하며, 일반적인 구분자(예: 줄바꿈)를 사용합니다.
from langchain_text_splitters import RecursiveCharacterTextSplitter

# RecursiveCharacterTextSplitter 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,              # 각 청크(chunk)의 최대 문자 수
    chunk_overlap=100,           # 청크 간 중첩(overlap) 문자 수
    add_start_index=True        # 각 청크의 시작 인덱스를 메타데이터로 추가
)

# 문서를 분할합니다. 'docs'는 분할할 원본 문서 목록입니다.
all_splits = text_splitter.split_documents([docs])

# 분할된 문서의 총 개수
len(all_splits)

5

In [27]:
# 첫 번째 청크(all_splits[0])의 내용을 벡터로 임베딩합니다.
vector_1 = embeddings.embed_query(all_splits[0].page_content)

# 두 번째 청크(all_splits[1])의 내용을 벡터로 임베딩합니다.
vector_2 = embeddings.embed_query(all_splits[1].page_content)

# 두 벡터의 길이가 동일한지 확인합니다.
assert len(vector_1) == len(vector_2), "벡터 길이가 일치하지 않습니다!"

# 벡터의 길이를 출력합니다.
print(f"생성된 벡터의 길이: {len(vector_1)}\n")

# 첫 번째 벡터의 일부(처음 10개 요소)를 출력합니다.
print(vector_1[:10])

생성된 벡터의 길이: 1536

[0.042093947529792786, -0.001852709916420281, -0.034876760095357895, 0.03672828897833824, 0.04126264899969101, -0.041149288415908813, 0.0011849550064653158, 0.07315429300069809, -0.011959366500377655, -0.003913244232535362]


### 벡터 스토어를 이용한 유사도 검색

`persist_directory=` 지정하면 disk folder 에 저장. 지정 않으면 in-memory 저장.

In [28]:
from langchain_chroma import Chroma

# 'embedding_function' 매개변수에 임베딩 함수를 전달하여 텍스트를 벡터로 변환합니다.
vector_store = Chroma(embedding_function=embeddings)
vector_store

In [29]:
# 주어진 문서 목록을 벡터 스토어에 추가하고 고유 식별자(ID) 반환
ids = vector_store.add_documents(documents=all_splits)
ids

['f0c29bc0-619f-42b2-80f9-802d97f96ac5',
 '0cf9df68-8b13-4a09-aacd-590045c3186e',
 '74cad254-52b8-4759-84ae-ac2ecf49cb29',
 '2df4c379-17d2-47c2-9ee0-5de065bd49d3',
 '56d37506-92c6-48b7-9a73-77f5bd784d50']

In [30]:
# 유사도 검색 수행
results = vector_store.similarity_search("인터넷이 준 영향?")

print(results)

[Document(metadata={'start_index': 901}, page_content='통신 및 연결성:\n컴퓨터는 통신에 혁명을 일으켜 사람들이 먼 거리에 있는 사람들과 연결할 수 있게 했습니다. 컴퓨터의 글로벌 네트워크인 인터넷은 이메일, 메시징 애플리케이션, 소셜 미디어 플랫폼을 통해 즉각적인 커뮤니케이션을 용이하게 했습니다. 그것은 우리가 정보를 공유하고, 지식에 접근하고, 온라인 커뮤니티에 참여하는 방식을 변화시켰습니다. 또한 화상 회의와 가상 현실의 발전은 지리적 격차를 메우고 몰입적이고 실시간 상호 작용을 가능하게 했습니다.\n\n교육 및 학습:\n컴퓨터는 교육에 큰 영향을 미쳐 우리가 배우고 지식을 습득하는 방식에 혁명을 일으켰습니다. 온라인 학습 플랫폼, 디지털 교과서, 교육 소프트웨어는 접근 가능하고 대화형 학습 경험을 제공합니다. 가상 시뮬레이션과 증강 현실 애플리케이션은 과학, 역사, 의학과 같은 과목에 대한 이해를 향상시킵니다. 게다가 컴퓨터는 교육에 대한 접근성을 확대하여 전 세계 개인에게 원격 학습 및 원격 교육 기회를 제공했습니다.'), Document(metadata={'start_index': 1394}, page_content='엔터테인먼트 및 창의성:\n컴퓨터는 엔터테인먼트 산업을 변화시켜 다양한 형태의 미디어를 만들고 소비할 수 있게 했습니다. 비디오 게임과 디지털 아트에서 음악 제작과 영화 편집에 이르기까지 컴퓨터는 창의적인 표현에 없어서는 안 될 도구가 되었습니다. 스트리밍 서비스와 온라인 플랫폼은 콘텐츠 배포를 민주화하여 전 세계 청중에게 다양한 엔터테인먼트 옵션을 제공합니다. 가상 현실과 증강 현실 기술은 몰입형 경험을 제공하여 디지털과 물리적 영역 간의 경계를 모호하게 합니다.'), Document(metadata={'start_index': 419}, page_content='컴퓨팅 파워의 진화:\n방 크기의 메인프레임에서 휴대용 노트북에 이르기까지 컴퓨터는 크기, 속도, 처리 능력 면에서 기하급수적으

In [31]:
results = vector_store.similarity_search("컴퓨팅 파워의 진화?")

print(results)

[Document(metadata={'start_index': 419}, page_content='컴퓨팅 파워의 진화:\n방 크기의 메인프레임에서 휴대용 노트북에 이르기까지 컴퓨터는 크기, 속도, 처리 능력 면에서 기하급수적으로 진화했습니다. 집적 회로, 마이크로프로세서, 마이크로칩의 트랜지스터 수가 약 2년마다 두 배가 된다는 무어의 법칙이 도입되면서 컴퓨터 기술이 발전했습니다. 이러한 기하급수적 성장으로 인해 더 빠르고 효율적이며 점점 더 강력한 장치가 개발되어 계속해서 세상을 재편하고 있습니다.\n\n업무와 생산성의 변화:\n컴퓨터는 산업 전반에 걸쳐 생산성, 효율성, 정확성을 높여 직장을 변화시켰습니다. 반복적인 작업을 자동화하고 운영을 간소화하며 글로벌 연결을 용이하게 했습니다. 워드 프로세싱과 데이터 분석에서 복잡한 시뮬레이션과 인공 지능에 이르기까지 컴퓨터는 금융, 의료, 엔지니어링, 창작 예술과 같은 분야의 전문가에게 필수적인 도구가 되었습니다. 원격 작업과 디지털 협업의 출현은 현대 작업 환경에 미치는 컴퓨터의 영향을 더욱 강조합니다.'), Document(metadata={'start_index': 1657}, page_content='결론:\n컴퓨터는 우리의 현대 세계를 근본적으로 형성하여 우리가 일하고, 소통하고, 배우고, 즐기는 방식에 혁명을 일으켰습니다. 부피가 큰 계산기에서 엄청난 처리 능력을 갖춘 휴대용 기기로의 진화는 인간의 혁신에 대한 증거입니다. 컴퓨터가 계속 발전함에 따라 사회적 진보를 주도하고, 복잡한 문제를 해결하고, 새로운 창의성의 길을 열어줄 수 있는 잠재력은 무한합니다. 책임감 있는 개발과 윤리적 사용을 통해 컴퓨터는 의심할 여지 없이 기술 발전의 최전선에 서서 개인에게 힘을 실어주고 미래 세대를 위해 사회를 변화시킬 것입니다.'), Document(metadata={'start_index': 901}, page_content='통신 및 연결성:\n컴퓨터는 통신에 혁명을 일으켜 사람들이 먼 거리에 있는 사람들과 연결